In [3]:
import pandas as pd
import numpy as np

# 1) Load
data = pd.read_csv("vgsales.csv")

# 2) Standardize column names
data.columns = (
        data.columns.str.strip()
                  .str.replace(r"[^\w]+", "_", regex=True)
                  .str.lower()
    )

# 3) Trim whitespace and normalize casing for text columns
text_cols = ["name", "platform", "genre", "publisher"]
for c in text_cols:
    if c in data.columns:
      data[c] = (data[c].astype(str)
                          .str.strip()
                          .str.replace(r"\s+", " ", regex=True))

#Title-case for readability (avoid if you do exact matching elsewhere)
    for c in ["name", "genre", "publisher"]:
        if c in data.columns:
            data[c] = data[c].str.title()
    if "platform" in data.columns:
        data["platform"] = data["platform"].str.upper()

  #4) Coerce Year to nullable integer (Int64), clamp to plausible range
    if "year" in data.columns:
        data["year"] = pd.to_numeric(data["year"], errors="coerce")
        # Many rows have NaN; keep as NaN for now
        data.loc[(data["year"] < 1970) | (data["year"] > pd.Timestamp.now().year+1), "year"] = np.nan
        data["year"] = data["year"].round().astype("Int64")  # round floats to nearest year
    data['year'].isnull().sum()
    if "year" in data.columns:
        median=data["year"].median()
        data["year"]=data["year"].fillna(median)


  # 5) Ensure sales columns are numeric and non-negative
    sales_cols = ["na_sales", "eu_sales", "jp_sales", "other_sales", "global_sales"]
    for c in sales_cols:
        if c in data.columns:
            data[c] = pd.to_numeric(data[c], errors="coerce")
            data.loc[data[c] < 0, c] = np.nan  # negative sales are invalid

  # 6) Recalculate global_sales if regionals exist
    have_regionals = all(c in data.columns for c in ["na_sales", "eu_sales", "jp_sales", "other_sales","global_sales"])
    if have_regionals:
        data["regional_sum"] = data[["na_sales", "eu_sales", "jp_sales", "other_sales","global_sales"]].sum(axis=1, skipna=True)
        # If global_sales missing or inconsistent by > 0.01, replace with regional sum
        if "global_sales" in data.columns:
            inconsistent = data["global_sales"].isna() | (data["global_sales"] - data["regional_sum"]).abs() > 0.01
            data.loc[inconsistent, "global_sales"] = data.loc[inconsistent, "regional_sum"]
        else:
            data["global_sales"] = data["regional_sum"]
        data.drop(columns=["regional_sum"], inplace=True)

    # 7) Drop exact duplicates
    key_cols = [c for c in ["name", "platform", "year", "publisher"] if c in data.columns]
    if key_cols:
        data = data.drop_duplicates(subset=key_cols, keep="first")
    else:
        data = data.drop_duplicates(keep="first")

    # 8) Remove rows with no sales at all (optional)
    if have_regionals:
        no_sales = ((data[["na_sales", "eu_sales", "jp_sales", "other_sales"]]).fillna(0).sum(axis=1) == 0)
        data = data.loc[~no_sales].copy()

    # 9) Standardize categorical values (light normalization)
    genre_map = {
        "Role-Playing": "Role-Playing",
        "Role Playing": "Role-Playing",
        "Rpg": "Role-Playing",
        "Sports": "Sports",
        "Action": "Action",
        "Shooter": "Shooter",
        "Platform": "Platform",
        "Racing": "Racing",
        "Puzzle": "Puzzle",
        "Misc": "Misc",
        "Simulation": "Simulation",
        "Adventure": "Adventure",
        "Fighting": "Fighting",
        "Strategy": "Strategy",
    }
    if "genre" in data.columns:
        data["genre"] = data["genre"].map(lambda x: genre_map.get(x, x))

    # Example: platform collapsing (optional, customize)
    data["platform_group"] = data["platform"].replace({
        "X360": "XBOX",
         "XB": "XBOX",
         "XONE": "XBOX",
        "PC":"PLAYSTATION",
        "PS": "PLAYSTATION",
        "PS2": "PLAYSTATION",
        "PS3": "PLAYSTATION",
        "PS4": "PLAYSTATION",
        "PSV": "PLAYSTATION",
        "PSP": "PLAYSTATION",
        "3DO":"PLAYSTATION",
        "GG": "PLAYSTATION",
        "PCFX":"PLAYSTATION",
        "DC":"PLAYSTATION",
        "GB": "NINTENDO",
        "GBA": "NINTENDO",
        "N64": "NINTENDO",
         "GC": "NINTENDO",
         "WII": "NINTENDO",
         "WIIU": "NINTENDO",
        "2600":"NINTENDO",
         "3DS": "NINTENDO",
         "DS": "NINTENDO",
       "NS": "NINTENDO",
        "NG":"NINTENDO",
       "NES":"NINTENDO",
        "SNES":"NINTENDO",
        "SAT":"NINTENDO",
        "WS":"NINTENDO",
        "TG16":"NINTENDO",
        "SCD":"NINTENDO",
        "GEN":"NINTENDO"
     })

    # 10) Create helpful features
    # Decade
    if "year" in data.columns:
        data["decade"] = data["year"].apply(lambda y: pd.NA if pd.isna(y) else int(y // 10 * 10)).astype("Int64")
    # Regional shares
    if have_regionals:
        total = data[["na_sales", "eu_sales", "jp_sales", "other_sales"]].sum(axis=1)
        for c in ["na_sales", "eu_sales", "jp_sales", "other_sales"]:
            data[c.replace("_sales", "_share")] = np.where(
                total > 0, data[c] / total, np.nan
                  )

    # 11) Final column order (optional)
    preferred_order = [c for c in [
        "name", "platform", "year", "decade", "genre", "publisher",
        "na_sales", "eu_sales", "jp_sales", "other_sales", "global_sales",
        "na_share", "eu_share", "jp_share", "other_share"
             ] if c in data.columns]
    data = data[[*preferred_order, *[c for c in data.columns if c not in preferred_order]]]
    data.to_csv('sorted.csv', index=False)